In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import math
import pylab
from scipy.interpolate import interp1d
import time
%matplotlib notebook

In [ ]:
#need lwa software library for this
from driftcurve import driftcurve, single_time_driftcurve
import cr_data_inspection_functions as crd

In [ ]:
#driftcurve(do_plot=True)

# part 1 : load up the data

In [ ]:
fname = '/scr/kp/cr_data/overnight_scan_20-21April2023/overnight_software_snapshots1682051758.6163533.dat'
events = crd.parsefile(fname, end_ind = 1000)
ant_id = np.array([events[i]['antenna_id'] for i in range(len(events))])
all_data_times = np.array([events[i]['timestamp'] for i in range(len(events))])
all_data_wfs = np.array([events[i]['data'] for i in range(len(events))])

In [ ]:
#select only one antenna
which_antenna = 0
cut = ant_id == 0
data_times = all_data_times[cut]
data_wfs = all_data_wfs[cut]
print('number of events:', len(data_times))

In [ ]:
def hanning_window(L):
    n = np.arange(L)
    return 0.5*(1-np.cos(2*np.pi*n/L))

def do_fft(fs, samples):
    '''
    returns only the positive frequencies of the FFT
    no normalization is applied
    applies a Hanning window to the FFT
    '''
    L = samples.shape[-1]

    N2 = L//2
    f_res = fs/L   # Frequency resolution

    Y = np.fft.fft(samples*hanning_window(L), L)

    # Get the values of interest
    Y_amp = np.abs(Y)
    Y_phase = np.arctan2(Y.imag, Y.real)

    freq = np.arange(0, N2)*f_res

    amp = Y_amp.T[:N2].T
    ph = Y_phase.T[:N2].T

    return freq, amp, ph

In [ ]:
N = data_wfs.shape[-1]
ADC_MAX = 2e9 #10 bits, but signed
ADC_to_V = 1.35 #TODO this is completely incorrect
fs = 196.e6 #approximate clock frequency
broadband_corr = 1.63 #correction for Hanning window
all_data_freq, data_amp, data_phase = do_fft(fs, data_wfs)
data_amp /= N
# next two numbers are to match with lwa paper
FREQ_MIN = 30.e6
FREQ_MAX = 80.e6
data_power = np.abs(data_amp)**2
all_P_data = 2. * broadband_corr**2 * (ADC_to_V/ADC_MAX)**2 * data_power #in rms V^2, factor of 2 is for negative freqs

#setting coax cable distance to 100m for now, will eventually vary with antenna
coax_distance = 100.

In [ ]:
plt.figure()
plt.semilogy(all_data_freq/1e6, np.mean(all_P_data,axis=0))

In [ ]:
#cut extra frequencies
FREQ_MIN = 30.e6
FREQ_MAX = 80.e6
cut = np.logical_and(all_data_freq >= FREQ_MIN, all_data_freq <= FREQ_MAX)
data_freq = all_data_freq[cut]
data_amp = data_amp[:,cut]
data_phase = data_phase[:,cut]
data_power = np.abs(data_amp)**2
P_data = 2. * broadband_corr**2 * (ADC_to_V/ADC_MAX)**2 * data_power #in rms V^2, factor of 2 is for negative freqs

#downsample frequencies to make this run faster
downsample_N = 50
P_data = P_data[:,::downsample_N]
data_freq =data_freq[::downsample_N]

# part 2 : set up the system of equations

In [ ]:
#gives skytemp in K and lst
#note that skytemp does not include antenna LNA gain
#lst, skytemp = driftcurve(do_plot=False)
dfreq = data_freq[1] - data_freq[0]
P_sim = []
k_b = 1.38e-23 #boltzmann const
r_ant = 50. # ohms, this is a guess
first = True
for t in data_times:
    Tsky = []
    for f in data_freq:
        lst, T = single_time_driftcurve(frequency = f)
        Tsky.append(T)
    first = False
    Tsky = np.array(Tsky)
    Psky = k_b * Tsky * dfreq * r_ant #in rms V^2
    P_sim.append(Psky)
P_sim = np.array(P_sim).reshape(data_times.shape[0], data_freq.shape[0])


In [ ]:
#need to load up antenna + LNA gain simulated
#this is what you need for Psim1 in lwa paper (https://arxiv.org/pdf/1903.05988.pdf)
#Psim1 = (Psky + N_lna) * G_ant*A_corr
#currently loads up gain digitized from fig 2 in the paper

#be careful, all of these interpolations have extrapolation on, which doesn't always work nicely

def load_antenna_gain(freq):
    fMHz, gdB = np.loadtxt('data/lwa_antenna_gain.csv', delimiter=',').T
    gfunc = interp1d(fMHz, gdB, fill_value='extrapolate')
    gain_out = gfunc(freq/1e6)
    return 10.**(gain_out/10.)

G_ant = load_antenna_gain(data_freq)

In [ ]:
#next you need to load up simulated coax cable losses to get
#Psim2 = (Psky + N_lna) * G_ant*A_corr*L_coax
#currently loads up gain/100m digitized from fig 2 in the paper

def load_coax_losses(freq, distance):
    fMHz, gdB = np.loadtxt('data/lwa_coax_per_100m.csv', delimiter=',').T
    gfunc = interp1d(fMHz, gdB, fill_value='extrapolate')
    gain_out = gfunc(freq/1e6) * distance/100.
    return 10.**(gain_out/10.)

L_coax = load_coax_losses(data_freq, coax_distance)

In [ ]:
#next you need to add in the last stage amplifier + ADC noise
#Psim3 = [(Psky + N_lna) * G_ant*A_corr*L_coax + N_RCU]*G_RCU*S_const + N_ADC
#currently loads up gain digitized from fig 3 in the paper

def load_RCU_gain(freq):
    fMHz, gdB = np.loadtxt('data/lwa_rcu_gain.csv', delimiter=',').T
    gfunc = interp1d(fMHz, gdB, fill_value='extrapolate')
    gain_out = gfunc(freq/1e6)
    return 10.**(gain_out/10.)

G_RCU = load_RCU_gain(data_freq)
plt.figure()
plt.plot(data_freq/1e6, 10.*np.log10(G_ant), label='antenna gain')
plt.plot(data_freq/1e6, 10.*np.log10(L_coax), label='coax loss')
plt.plot(data_freq/1e6, 10.*np.log10(G_RCU), label='filter gain')
plt.xlabel("frequency (MHz)")
plt.ylabel('gain (dB)')
plt.legend()

# part 3 : use system of equations to find unknowns

In [ ]:
from scipy.optimize import curve_fit
#note that curve fit needs things flattened

def optfunc(x, N_lna, N_RCU, S_const, N_ADC, *A_corr):
    # G_ant, L_coax and G_RCU are implicit global here (okay for now)
    A_i = np.array(A_corr)
    y = ((x + N_lna)*G_ant*A_i*L_coax + N_RCU)*G_RCU*S_const + N_ADC
    #y = ((x + N_lna)*G_ant*L_coax + N_RCU)*G_RCU*S_const + N_ADC #removed the antenna correction factor
    return y.flatten()

popt, pcov = curve_fit(optfunc, P_sim, P_data.flatten(), p0 = np.ones(4 + data_freq.shape[0]), bounds = (0, np.inf))

In [ ]:
cal_factor = np.sqrt(1/(popt[4:] * L_coax * G_RCU * popt[2])) # C**2 = (A_corr * L_coax * G_RCU * S)**-1
print(cal_factor)

In [ ]:
print(popt)

In [ ]:
plt.figure()
plt.semilogy(data_freq/1e6, np.mean(P_sim, axis=0)/(50*np.diff(data_freq/1e6)[0]), label='sky_noise')
plt.legend()
plt.xlabel('frequency (MHz)')
plt.ylabel('noise spectral density (watts/MHz)')